In [7]:
import numpy as np
import pandas as pd
import config.config_paths as paths
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

Data loading and exploration

In [8]:
# Reading the CSV files
calendar_df = pd.read_csv(paths.calendar_path)
sales_train_evaluation_df = pd.read_csv(paths.sales_train_eval_path)
sell_prices_df = pd.read_csv(paths.sell_prices_path)

In [9]:
def downcast_types(df):
    float_cols = [c for c in df if df[c].dtype == 'float64']
    int_cols = [c for c in df if df[c].dtype in ['int64', 'int32']]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

In [10]:
calendar_df = downcast_types(calendar_df)
calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          1969 non-null   object
 1   wm_yr_wk      1969 non-null   int16 
 2   weekday       1969 non-null   object
 3   wday          1969 non-null   int16 
 4   month         1969 non-null   int16 
 5   year          1969 non-null   int16 
 6   d             1969 non-null   object
 7   event_name_1  162 non-null    object
 8   event_type_1  162 non-null    object
 9   event_name_2  5 non-null      object
 10  event_type_2  5 non-null      object
 11  snap_CA       1969 non-null   int16 
 12  snap_TX       1969 non-null   int16 
 13  snap_WI       1969 non-null   int16 
dtypes: int16(7), object(7)
memory usage: 134.7+ KB


In [11]:
sales_train_evaluation_df = downcast_types(sales_train_evaluation_df)
sales_train_evaluation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1947 entries, id to d_1941
dtypes: int16(1941), object(6)
memory usage: 114.3+ MB


In [12]:
sales_train_evaluation_df.head(10)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
5,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,0,0,1,0,0,5,2,0
6,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,0,0,0,1,0,1,1,0
7,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,12,15,0,0,...,7,0,6,0,15,5,4,1,40,32
8,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,2,0,7,3,...,1,0,0,0,0,0,0,0,1,0
9,HOBBIES_1_010_CA_1_evaluation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,...,0,0,1,0,2,1,1,0,0,1


In [13]:
# Display basic statistics for numerical columns
print("Summary statistics:\n", calendar_df.describe())

# Count missing (null) values for each column
missing_values = calendar_df.isnull().sum()
print("\nMissing values per column:\n", missing_values)

Summary statistics:
            wm_yr_wk         wday        month         year      snap_CA  \
count   1969.000000  1969.000000  1969.000000  1969.000000  1969.000000   
mean   11347.086338     3.997461     6.325546  2013.288471     0.330117   
std      155.277043     2.001141     3.416864     1.580198     0.470374   
min    11101.000000     1.000000     1.000000  2011.000000     0.000000   
25%    11219.000000     2.000000     3.000000  2012.000000     0.000000   
50%    11337.000000     4.000000     6.000000  2013.000000     0.000000   
75%    11502.000000     6.000000     9.000000  2015.000000     1.000000   
max    11621.000000     7.000000    12.000000  2016.000000     1.000000   

           snap_TX      snap_WI  
count  1969.000000  1969.000000  
mean      0.330117     0.330117  
std       0.470374     0.470374  
min       0.000000     0.000000  
25%       0.000000     0.000000  
50%       0.000000     0.000000  
75%       1.000000     1.000000  
max       1.000000     1.000000

In [14]:
## Check and fill missing values
calendar_df.fillna("unknown", inplace=True)

In [15]:
# Feature Engineering
sales_data = sales_train_evaluation_df.melt(
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='d',
    value_name='daily_sales'
)
sales_data.head(10)

,id,item_id,dept_id,cat_id,store_id,state_id,d,daily_sales
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
5,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
6,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
7,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12
8,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2
9,HOBBIES_1_010_CA_1_evaluation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [16]:
## Merge calendar events into sales data
# sales_data.drop(columns='id', inplace=True)
sales_data = pd.merge(sales_data, calendar_df[
    ['date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'd', 'event_name_1', 'event_type_1', 'event_name_2',
     'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI']], on='d')

In [17]:
## Add sell_prices information
sales_data = pd.merge(sales_data, sell_prices_df, on=['store_id', 'item_id', 'wm_yr_wk'])

In [18]:
sales_data.head(10)

,id,item_id,dept_id,cat_id,store_id,state_id,d,daily_sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,2011-01-29,11101,...,1,2011,unknown,unknown,unknown,unknown,0,0,0,0.46
1,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_2,15,2011-01-30,11101,...,1,2011,unknown,unknown,unknown,unknown,0,0,0,0.46
2,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0,2011-01-31,11101,...,1,2011,unknown,unknown,unknown,unknown,0,0,0,0.46
3,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0,2011-02-01,11101,...,2,2011,unknown,unknown,unknown,unknown,1,1,0,0.46
4,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0,2011-02-02,11101,...,2,2011,unknown,unknown,unknown,unknown,1,0,1,0.46
5,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_6,4,2011-02-03,11101,...,2,2011,unknown,unknown,unknown,unknown,1,1,1,0.46
6,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_7,6,2011-02-04,11101,...,2,2011,unknown,unknown,unknown,unknown,1,0,0,0.46
7,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2,2011-01-29,11101,...,1,2011,unknown,unknown,unknown,unknown,0,0,0,1.56
8,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_2,0,2011-01-30,11101,...,1,2011,unknown,unknown,unknown,unknown,0,0,0,1.56
9,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_3,7,2011-01-31,11101,...,1,2011,unknown,unknown,unknown,unknown,0,0,0,1.56


In [19]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46881677 entries, 0 to 46881676
Data columns (total 22 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id            object 
 1   item_id       object 
 2   dept_id       object 
 3   cat_id        object 
 4   store_id      object 
 5   state_id      object 
 6   d             object 
 7   daily_sales   int16  
 8   date          object 
 9   wm_yr_wk      int16  
 10  weekday       object 
 11  wday          int16  
 12  month         int16  
 13  year          int16  
 14  event_name_1  object 
 15  event_type_1  object 
 16  event_name_2  object 
 17  event_type_2  object 
 18  snap_CA       int16  
 19  snap_TX       int16  
 20  snap_WI       int16  
 21  sell_price    float64
dtypes: float64(1), int16(8), object(13)
memory usage: 5.6+ GB


In [20]:
## remove non-relevant features for LSTM input
sales_data.drop(columns=['d', 'date', 'weekday'], inplace=True)
sales_data = downcast_types(sales_data)
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46881677 entries, 0 to 46881676
Data columns (total 19 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id            object 
 1   item_id       object 
 2   dept_id       object 
 3   cat_id        object 
 4   store_id      object 
 5   state_id      object 
 6   daily_sales   int16  
 7   wm_yr_wk      int16  
 8   wday          int16  
 9   month         int16  
 10  year          int16  
 11  event_name_1  object 
 12  event_type_1  object 
 13  event_name_2  object 
 14  event_type_2  object 
 15  snap_CA       int16  
 16  snap_TX       int16  
 17  snap_WI       int16  
 18  sell_price    float32
dtypes: float32(1), int16(8), object(10)
memory usage: 4.4+ GB


In [21]:
sales_data.head(10)

,id,item_id,dept_id,cat_id,store_id,state_id,daily_sales,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,12,11101,1,1,2011,unknown,unknown,unknown,unknown,0,0,0,0.46
1,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,15,11101,2,1,2011,unknown,unknown,unknown,unknown,0,0,0,0.46
2,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,0,11101,3,1,2011,unknown,unknown,unknown,unknown,0,0,0,0.46
3,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,0,11101,4,2,2011,unknown,unknown,unknown,unknown,1,1,0,0.46
4,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,0,11101,5,2,2011,unknown,unknown,unknown,unknown,1,0,1,0.46
5,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,4,11101,6,2,2011,unknown,unknown,unknown,unknown,1,1,1,0.46
6,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,6,11101,7,2,2011,unknown,unknown,unknown,unknown,1,0,0,0.46
7,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,2,11101,1,1,2011,unknown,unknown,unknown,unknown,0,0,0,1.56
8,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,0,11101,2,1,2011,unknown,unknown,unknown,unknown,0,0,0,1.56
9,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,7,11101,3,1,2011,unknown,unknown,unknown,unknown,0,0,0,1.56


In [22]:
all_items = sales_data['id'].unique()

In [23]:
item0_df = sales_data[sales_data['id'] == all_items[0]]

In [24]:
item_sales_df = item0_df[
    ['wm_yr_wk', 'wday', 'month', 'year', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA',
     'snap_TX', 'snap_TX', 'sell_price', 'daily_sales']].reset_index(drop=True)
item_sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1941 entries, 0 to 1940
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   wm_yr_wk      1941 non-null   int16  
 1   wday          1941 non-null   int16  
 2   month         1941 non-null   int16  
 3   year          1941 non-null   int16  
 4   event_name_1  1941 non-null   object 
 5   event_type_1  1941 non-null   object 
 6   event_name_2  1941 non-null   object 
 7   event_type_2  1941 non-null   object 
 8   snap_CA       1941 non-null   int16  
 9   snap_TX       1941 non-null   int16  
 10  snap_TX       1941 non-null   int16  
 11  sell_price    1941 non-null   float32
 12  daily_sales   1941 non-null   int16  
dtypes: float32(1), int16(8), object(4)
memory usage: 98.7+ KB


In [25]:
item_sales_df.count()

wm_yr_wk        1941
wday            1941
month           1941
year            1941
event_name_1    1941
event_type_1    1941
event_name_2    1941
event_type_2    1941
snap_CA         1941
snap_TX         1941
snap_TX         1941
sell_price      1941
daily_sales     1941
dtype: int64

In [26]:
scaler = MinMaxScaler()
numerical_features = ['daily_sales', 'sell_price']
item_sales_df[numerical_features] = scaler.fit_transform(item_sales_df[numerical_features])

In [27]:
# Perform one-hot encoding for the specified columns
one_hot_encoded_columns = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
item_sales_df = pd.get_dummies(item_sales_df, columns=one_hot_encoded_columns)

In [28]:
bool_cols = item_sales_df.select_dtypes(include=['bool']).columns.tolist()
for col in bool_cols:
    item_sales_df[col] = item_sales_df[col].astype(int)
int32_cols = item_sales_df.select_dtypes(include=['int32']).columns.tolist()
for col in int32_cols:
    item_sales_df[col] = item_sales_df[col].astype('int16')

item_sales_df.head(200)

,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_TX,sell_price,daily_sales,event_name_1_Chanukah End,...,event_type_1_Sporting,event_type_1_unknown,event_name_2_Cinco De Mayo,event_name_2_Easter,event_name_2_Father's day,event_name_2_OrthodoxEaster,event_name_2_unknown,event_type_2_Cultural,event_type_2_Religious,event_type_2_unknown
0,11101,1,1,2011,0,0,0,0.5,0.131868,0,...,0,1,0,0,0,0,1,0,0,1
1,11101,2,1,2011,0,0,0,0.5,0.164835,0,...,0,1,0,0,0,0,1,0,0,1
2,11101,3,1,2011,0,0,0,0.5,0.000000,0,...,0,1,0,0,0,0,1,0,0,1
3,11101,4,2,2011,1,1,1,0.5,0.000000,0,...,0,1,0,0,0,0,1,0,0,1
4,11101,5,2,2011,1,0,0,0.5,0.000000,0,...,0,1,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,11128,7,8,2011,0,1,1,1.0,0.043956,0,...,0,1,0,0,0,0,1,0,0,1
196,11129,1,8,2011,0,1,1,1.0,0.043956,0,...,0,1,0,0,0,0,1,0,0,1
197,11129,2,8,2011,0,0,0,1.0,0.109890,0,...,0,1,0,0,0,0,1,0,0,1
198,11129,3,8,2011,0,1,1,1.0,0.021978,0,...,0,1,0,0,0,0,1,0,0,1


In [29]:
item_sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1941 entries, 0 to 1940
Data columns (total 53 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   wm_yr_wk                          1941 non-null   int16  
 1   wday                              1941 non-null   int16  
 2   month                             1941 non-null   int16  
 3   year                              1941 non-null   int16  
 4   snap_CA                           1941 non-null   int16  
 5   snap_TX                           1941 non-null   int16  
 6   snap_TX                           1941 non-null   int16  
 7   sell_price                        1941 non-null   float32
 8   daily_sales                       1941 non-null   float32
 9   event_name_1_Chanukah End         1941 non-null   int16  
 10  event_name_1_Christmas            1941 non-null   int16  
 11  event_name_1_Cinco De Mayo        1941 non-null   int16  
 12  event_

In [30]:
item_sales_df.shape

(1941, 53)

In [31]:
# for i in range(n_past, len(item_sales_df) - n_future + 1):


In [32]:
## Reshape data for LSTM input format [samples, time steps, features]
# time_window = 14
# data = []
# for i in range(len(sales_data) - time_window):
#     data.append(sales_data.iloc[i:i+time_window].values)
# data = np.array(data)

In [33]:
# def print_array_info(array):
#     print("Array Elements:")
#     print(array)
#     print("Array Shape:", array.shape)
#     print("Array Size:", array.size)
#     print("Array Dimensions:", array.ndim)
#     print("Array Data Type:", array.dtype)
#
# print_array_info(data)

In [35]:
# The expected output is a DataFrame containing the predicted sales for the next 28 days (F1-F28) for each item-store pair, both for the public leaderboard (validation) and private leaderboard (evaluation).
#
#
# For evaluating the model, we used the RMSSE score as specified. It measures the deviation between predicted sales and actual sales at the store level, taking into account the scaling factor of actual sales. This provides a better understanding of forecast accuracy and helps in comparing models.